#### Univariate Analysis

The ADVATAGE of using this method of random sample imputation is that we are selecting random number from the data to fill the missing data. Hence the  DISTRIBUTION of DATA does not change. 

Why is it that the distribution of data does not change that is because. 

- When we are picking up random values from the dataset we are picking up the values from the central area(Imagin a normal distribution) where the values are the most frequent.  


This method works good for Linear Models but not so much for Tree models. 

In [154]:
import polars as pl
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

In [155]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from scipy.stats import gaussian_kde
from sklearn.model_selection import train_test_split

In [156]:
df = pl.read_csv(
    'Data_set\\train_1.csv',
    null_values = 'NA',     # the is NA value in int column treat it as a missing value 
).select(['Age', 'Fare', 'Survived'])
df.head(2)

Age,Fare,Survived
f64,f64,i64
22.0,7.25,0
38.0,71.2833,1


In [ ]:
null_percentage = df.select([
    (pl.col(c).is_null().sum()/df.height)
    for c in df.columns
]) * 100

null_percentage = null_percentage.unpivot(
    index           = [],
    variable_name   ='column',
    value_name      = 'Percentage (%)'
)

null_percentage

column,Percentage (%)
str,f64
"""Age""",19.86532
"""Fare""",0.0
"""Survived""",0.0


In [158]:
x = df.drop('Survived')
y = df['Survived']

In [159]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=2)

In [160]:
x_train = x_train.with_columns(
    pl.col('Age').alias('Age_imputed')
)
x_test = x_train.with_columns(
    pl.col('Age').alias('Age_imputed')
)

In [161]:
#    replacing in the x_train column
non_null_age_train = x_train.select(pl.col('Age').drop_nulls()).to_series().to_numpy()   # selecting all the value of Age, dropping the nulls - this array will be used to randomly sample values for imputation
null_count = x_train.select(pl.col('Age_imputed').is_null().sum()).item()                # count the number of missing null value in Age_imputed column of x_train

# randomly selecting values from the non-null-age-train 
sampled_train = np.random.choice(non_null_age_train, null_count, replace = True)   

null_mask = x_train['Age_imputed'].is_null().to_numpy()       # true where the values are null

# Replace null values with sampled values
filled_values = x_train['Age_imputed'].to_numpy()
filled_values[null_mask] = sampled_train

x_train = x_train.with_columns([
    pl.Series('Age_imputed', filled_values)
])

#    replacing in the x_test column
non_null_age_test = x_test.select(pl.col('Age').drop_nulls()).to_series().to_numpy()
null_count = x_test.select(pl.col('Age_imputed').is_null().sum()).item()

sample_train = np.random.choice(non_null_age_train, null_count, replace = True)
null_mask = x_test['Age_imputed'].is_null().to_numpy()

filled_value = x_test['Age_imputed'].to_numpy()
filled_value[null_mask] = sample_train

x_test = x_test.with_columns([
    pl.Series('Age_imputed', filled_value)
])

In [162]:
non_null_age = x_train.select(pl.col('Age').drop_nulls()).to_series().to_numpy()
null_count = x_train.select(pl.col('Age').is_null().sum()).item()

sampled_values_train = np.random.choice(non_null_age, null_count, replace = True)

Plotting the dist plot

In [163]:
age_orig = x_train.select('Age').to_series().to_pandas().dropna()
age_imp  = x_train.select('Age_imputed').to_series().to_pandas().dropna()

x_vals = np.linspace(min(age_orig.min(), age_imp.min()),
                     max(age_orig.max(), age_imp.max()), 200)

kde_orig = gaussian_kde(age_orig)(x_vals)
kde_imp  = gaussian_kde(age_imp)(x_vals)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x_vals,
    y=kde_imp,
    name='Imputed',
    mode='lines',
    fill='tozeroy',
    line=dict(color='red'),
    fillcolor='rgba(255, 0, 0, 0.3)'
))

fig.add_trace(go.Scatter(
    x=x_vals,
    y=kde_orig,
    name='Original',
    mode='lines',
    fill='tozeroy',
    line=dict(color='blue'),
    fillcolor='rgba(0, 0, 255, 0.3)'
))

fig.update_layout(
    title='KDE of Age: Original vs Imputed',
    xaxis_title='Age',
    yaxis_title='Density',
    plot_bgcolor='white',
    yaxis=dict(showgrid=True, gridcolor='lightgray'),
    xaxis=dict(showgrid=False)
)

fig.show()

Important observation the Distrubution of the curve remain the same because the random values are picked up from the distribution itself. 

In [164]:
print('Original variable variance : ', x_train.select(pl.col('Age').var()).item())
print('variance after random imputation : ', x_train.select(pl.col('Age_imputed').var()).item())

Original variable variance :  204.34951339046142
variance after random imputation :  196.58996101530522


In [165]:
x_train.select(['Fare', 'Age', 'Age_imputed']).to_pandas().cov()

,Fare,Age,Age_imputed
Fare,2368.246832,71.512440,53.473283
Age,71.512440,204.349513,204.349513
Age_imputed,53.473283,204.349513,196.589961


There is a change in covariance when we use this technique

In [166]:
df_plot = x_train.select(['Age', 'Age_imputed']).to_pandas()
df_long = df_plot.melt(var_name = 'Variable', value_name = 'Value')

fig = px.box(df_long, x='Variable', y='Value', title = 'Boxplot of Age vs Age_imputed')
fig.update_layout(yaxis_title = 'Value', xaxis_title = 'Variable')
fig.show()

There is not much change in outlier detection when we use this technique

---

Working on categorical dataset

In [167]:
df = pl.read_csv(
    'Data_set\\house-train.csv',
    null_values = 'NA',     # the is NA value in int column treat it as a missing value 
).select(['GarageQual', 'FireplaceQu', 'SalePrice'])
df.head(2)

GarageQual,FireplaceQu,SalePrice
str,str,i64
"""TA""",null,208500
"""TA""","""TA""",181500


In [168]:
null_percentage = df.select([
    (pl.col(c).is_null().sum()/df.height)
    for c in df.columns
]) * 100

null_percentage = null_percentage.unpivot(
    index = [],
    variable_name='column',
    value_name = 'Percentage (%)'
)

null_percentage

column,Percentage (%)
str,f64
"""GarageQual""",5.547945
"""FireplaceQu""",47.260274
"""SalePrice""",0.0


In [169]:
x = df
y = df['SalePrice']

In [170]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

In [171]:
x_train = x_train.with_columns(
    pl.col('GarageQual').alias('GarageQual_imputed')
)
x_test = x_train.with_columns(
    pl.col('GarageQual').alias('GarageQual_imputed')
)

x_train = x_train.with_columns(
    pl.col('FireplaceQu').alias('FireplaceQu_imputed')
)
x_test = x_train.with_columns(
    pl.col('FireplaceQu').alias('FireplaceQu_imputed')
)

In [172]:
#    replacing in the x_train column
non_null_GarageQual_train = x_train.select(pl.col('GarageQual').drop_nulls()).to_series().to_numpy()   # selecting all the value of Age, dropping the nulls - this array will be used to randomly sample values for imputation
null_count = x_train.select(pl.col('GarageQual_imputed').is_null().sum()).item()                # count the number of missing null value in Age_imputed column of x_train

# randomly selecting values from the non-null-age-train 
sampled_train = np.random.choice(non_null_GarageQual_train, null_count, replace = True)   

null_mask = x_train['GarageQual_imputed'].is_null().to_numpy()       # true where the values are null

# Replace null values with sampled values
filled_values = x_train['GarageQual_imputed'].to_numpy()
filled_values[null_mask] = sampled_train

x_train = x_train.with_columns([
    pl.Series('GarageQual_imputed', filled_values)
])

#    replacing in the x_test column
non_null_GarageQual_test = x_test.select(pl.col('GarageQual').drop_nulls()).to_series().to_numpy()
null_count = x_test.select(pl.col('GarageQual_imputed').is_null().sum()).item()

sample_train = np.random.choice(non_null_GarageQual_train, null_count, replace = True)
null_mask = x_test['GarageQual_imputed'].is_null().to_numpy()

filled_value = x_test['GarageQual_imputed'].to_numpy()
filled_value[null_mask] = sample_train

x_test = x_test.with_columns([
    pl.Series('GarageQual_imputed', filled_value)
])

In [173]:
#    replacing in the x_train column
non_null_FireplaceQu_train = x_train.select(pl.col('FireplaceQu').drop_nulls()).to_series().to_numpy()   # selecting all the value of Age, dropping the nulls - this array will be used to randomly sample values for imputation
null_count = x_train.select(pl.col('FireplaceQu_imputed').is_null().sum()).item()                # count the number of missing null value in Age_imputed column of x_train

# randomly selecting values from the non-null-age-train 
sampled_train = np.random.choice(non_null_FireplaceQu_train, null_count, replace = True)   

null_mask = x_train['FireplaceQu_imputed'].is_null().to_numpy()       # true where the values are null

# Replace null values with sampled values
filled_values = x_train['FireplaceQu_imputed'].to_numpy()
filled_values[null_mask] = sampled_train

x_train = x_train.with_columns([
    pl.Series('FireplaceQu_imputed', filled_values)
])

#    replacing in the x_test column
non_null_age_test = x_test.select(pl.col('FireplaceQu').drop_nulls()).to_series().to_numpy()
null_count = x_test.select(pl.col('FireplaceQu_imputed').is_null().sum()).item()

sample_train = np.random.choice(non_null_FireplaceQu_train, null_count, replace = True)
null_mask = x_test['FireplaceQu_imputed'].is_null().to_numpy()

filled_value = x_test['FireplaceQu_imputed'].to_numpy()
filled_value[null_mask] = sample_train

x_test = x_test.with_columns([
    pl.Series('FireplaceQu_imputed', filled_value)
])

In [ ]:
# Get normalized value counts for both columns
original_counts = x_train.select(pl.col('GarageQual').drop_nulls()).to_pandas()['GarageQual'].value_counts()
imputed_counts = x_train.select(pl.col('GarageQual_imputed')).to_pandas()['GarageQual_imputed'].value_counts()

# Normalize
original = original_counts / len(x_train['GarageQual'].drop_nulls())
imputed = imputed_counts / len(x_train)

# Combine into a single DataFrame
temp = pd.concat([original, imputed], axis=1)
temp.columns = ['original', 'imputed']
temp

,original,imputed
TA,0.951043,0.952055
Fa,0.037171,0.035959
Gd,0.009973,0.010274
Po,0.000907,0.000856
Ex,0.000907,0.000856


In [175]:
original = (
    x_train.filter(pl.col("GarageQual").is_not_null())
           .group_by("GarageQual")
           .agg(pl.len().alias("original"))
           .with_columns((pl.col("original") / pl.col("original").sum()).alias("original"))
)

imputed = (
    x_train.group_by("GarageQual_imputed")
           .agg(pl.len().alias("imputed"))
           .with_columns((pl.col("imputed") / pl.col("imputed").sum()).alias("imputed"))
           .rename({"GarageQual_imputed": "GarageQual"})
)

result = original.join(imputed, on="GarageQual", how="full").sort("GarageQual")
result = result.drop(['GarageQual_right'])
result

GarageQual,original,imputed
str,f64,f64
"""Ex""",0.000907,0.000856
"""Fa""",0.037171,0.035959
"""Gd""",0.009973,0.010274
"""Po""",0.000907,0.000856
"""TA""",0.951043,0.952055


In [176]:
original = (
    x_train.filter(pl.col("FireplaceQu").is_not_null())
           .group_by("FireplaceQu")
           .agg(pl.len().alias("original"))
           .with_columns((pl.col("original") / pl.col("original").sum()).alias("original"))
)

imputed = (
    x_train.group_by("FireplaceQu_imputed")
           .agg(pl.len().alias("imputed"))
           .with_columns((pl.col("imputed") / pl.col("imputed").sum()).alias("imputed"))
           .rename({"FireplaceQu_imputed": "FireplaceQu"})
)

result = original.join(imputed, on="FireplaceQu", how="full").sort("FireplaceQu")
result = result.drop(['FireplaceQu_right'])
result

FireplaceQu,original,imputed
str,f64,f64
"""Ex""",0.02455,0.023116
"""Fa""",0.040917,0.04024
"""Gd""",0.494272,0.494863
"""Po""",0.027823,0.032534
"""TA""",0.412439,0.409247


In [184]:
fig = go.Figure()

categories = x_train.select(pl.col('FireplaceQu').drop_nulls()).to_series().unique()

for category in categories:
    subset = (
        x_train
        .filter(pl.col('FireplaceQu') == category)
        .select('SalePrice')
        .to_series()
        .drop_nulls()
        .to_numpy()
    )
    
    if len(subset) < 2:
        continue
    
    x_vals = np.linspace(subset.min(), subset.max(), 200)
    kde_vals = gaussian_kde(subset)(x_vals)

    fig.add_trace(go.Scatter(
        x=x_vals,
        y=kde_vals,
        name=str(category),
        mode='lines',
        fill='tozeroy'  # Use default fill with auto color
    ))

fig.update_layout(
    title='KDE of SalePrice by Fireplace Quality',
    xaxis_title='SalePrice',
    yaxis_title='Density',
    plot_bgcolor='white',
    yaxis=dict(showgrid=True, gridcolor='lightgray'),
    xaxis=dict(showgrid=False)
)

fig.show()


In [185]:
fig = go.Figure()

# Get unique non-null categories from the imputed column
categories = x_train.select(pl.col('FireplaceQu_imputed').drop_nulls()).to_series().unique()

for category in categories:
    # Filter SalePrice values where FireplaceQu_imputed equals the category using Polars
    subset = (
        x_train
        .filter(pl.col('FireplaceQu_imputed') == category)
        .select('SalePrice')
        .to_series()
        .drop_nulls()
        .to_numpy()
    )
    
    if len(subset) < 2:
        continue  # KDE requires at least 2 points

    x_vals = np.linspace(subset.min(), subset.max(), 200)
    kde_vals = gaussian_kde(subset)(x_vals)

    fig.add_trace(go.Scatter(
        x=x_vals,
        y=kde_vals,
        name=str(category),
        mode='lines',
        fill='tozeroy'  # Automatically colored and filled by Plotly
    ))

fig.update_layout(
    title='KDE of SalePrice by FireplaceQu_imputed',
    xaxis_title='SalePrice',
    yaxis_title='Density',
    plot_bgcolor='white',
    yaxis=dict(showgrid=True, gridcolor='lightgray'),
    xaxis=dict(showgrid=False)
)

fig.show()